In [3]:
# installing bayesian optimization 
!pip install GPy
!pip install GPyOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.4/959.4 kB 18.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPy: filename=GPy-1.10.0-cp37-cp37m-linux_x86_64.whl size=3119634 sha256=893a1545f82bcf3c53d6bdb29664fbb2d2dd6611efb94cb786f0c0c74b0d8c58
  Stored in directory: /root/.cache/pip/wheels/f7/18/28/dd1ce0192a81b71a3b086fd952511d088b21e8359ea496860a
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102566 sha256=d67a77d9f17730229f689d6a95d4f9ae3c640f5af35bcff94bffb9c8cb0a9f69
  Stored in directory: /root/.cache/pip/wheels/c8/95/f5/ce28482da28162e6028c4b3a32c41d147395825b3cd62bc810
Successfully built GPy paramz
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPyOpt: filename=GPyOpt-1.2.6-py3-none-any.whl size=83609 

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers
from keras.models import Sequential,Model
import keras.layers as kl
from keras.layers.merge import concatenate
import gc
import math
import time
import GPy, GPyOpt

In [5]:
# computing RMSE and SMAPE for single and multiple points
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def single_point_smape(actual, forecast):
    return (100/len(actual)) * (np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast))))
def smape(actual, forecast):
    sum_smape=0
    for i in range(len(actual)):
        sum_smape+=single_point_smape(actual[i],forecast[i])
    return sum_smape/len(actual)

In [7]:
# creating samples from raw time series
# look_forward is number of points we want to predict from each sample.
# look_back is simply lag size ( window size ). 
def create_dataset(dataset, look_back,look_forward=7):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-look_forward+1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        a=dataset[(i + look_back):(i + look_back+look_forward), 0]
        dataY.append(a)
    return np.array(dataX), np.array(dataY)

In [ ]:
# creating samples based on features 
def create_dataset_features(dataset, look_back,look_forward=7):
    sample = []
    sample = [float(item) for item in dataset[look_back-5] if math.isnan(item)==False]
    sample=np.array(sample)
    sample = sample.reshape(sample.shape[0],1)
    # number of features are 11, thus we reshape based on this
    sample=sample.reshape(11,int(len(sample)/11))
    sample=sample.T
    sample=np.array(sample)
    sample=sample[0:-look_forward]
    return sample

In [74]:
# here we need to split train, test, validation
def train_test_split(dataX,dataY,one_sample,look_back,look_forward):
    # test size should be selected manually, here we just select the points not number of samples.
    testsize=35
    train_size=(len(one_sample)-look_forward)-testsize-look_back+1
    
    trainX, testX = dataX[0:train_size,:], dataX[train_size+look_forward-1:,:]
    trainY, testY = dataY[0:train_size,:], dataY[train_size+look_forward-1:,:]
    # validation size should be selected manually, here we just select the points not number of samples.
    valsize=33

    valX, valY=trainX[train_size-valsize+look_forward-1:train_size,:],trainY[train_size-valsize+look_forward-1:train_size]

    trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
    valX= np.reshape(valX, (valX.shape[0],1, valX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))
    return np.array(trainX),np.array(valX),np.array(testX),np.array(trainY),np.array(valY),np.array(testY)

In [75]:
# reading the corresponding country csv file. this is confirmed cases dataset.
all_train = pd.read_csv('../input/indiacovid-reza-ali/us.csv')
all_train=np.array(all_train)
all_train

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
0,US,40.000000,-100.000000,1,2,5,7,8,11,12,...,4713540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brazil,-14.235000,-51.925300,1,2,4,13,20,25,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,India,20.593684,78.962880,1,2,3,5,28,30,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russia,61.524010,105.318756,2,3,4,13,17,20,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,South Africa,-30.559500,22.937500,1,3,7,13,17,24,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Mexico,23.634500,-102.552800,1,4,5,6,7,8,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Peru,-9.190000,-75.015200,1,6,7,11,15,28,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Chile,-35.675100,-71.543000,2,9,10,13,20,25,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Colombia,4.570900,-74.297300,1,3,9,13,22,34,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Iran,32.427908,53.688046,2,5,18,28,43,61,95,...,312035.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# reading the feature file.
features = pd.read_csv('/kaggle/input/us-covid19-with-feature/US_features.csv')
features=features.T
features=np.array(features)
features

,x,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10
0,4.6,5.666667,6.571429,7.375,8.222222,9.0,10.363636,12.000000,15.153846,19.285714,24.933333
1,6.6,7.500000,8.285714,9.125,9.888889,11.3,13.000000,16.333333,20.692308,26.642857,36.466667
2,8.6,9.333333,10.142857,10.875,12.333333,14.1,17.636364,22.250000,28.538462,38.928571,51.133333
3,10.2,11.000000,11.714286,13.250,15.111111,18.9,23.818182,30.500000,41.538462,54.428571,73.266667
4,11.8,12.500000,14.142857,16.125,20.222222,25.5,32.636364,44.416667,58.076923,78.000000,102.866667
...,...,...,...,...,...,...,...,...,...,...,...
1788,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1789,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1790,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1791,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
look_back=0
def final_sample_creation(one_sample,features,lag,look_forward,all_features,):
    # we should define the look_forward variable. this is the number of points we want to predict.
    # our problem is multi-variable prediction and in this dataset the output length is 7. 
    look_forward=7
    look_back=lag

    one_sample=np.array(one_sample)
    one_sample = one_sample.astype('int64')
    one_sample = one_sample.reshape(one_sample.shape[0],1)
    

    dataX_s, dataY_s = create_dataset(one_sample,look_back,look_forward)
    trainX,valX,testX,trainY,valY,testY = train_test_split(dataX_s,dataY_s,one_sample,look_back,look_forward)
    
    dataX_features = create_dataset_features(features,look_back,look_forward)
    dataX_features = dataX_features[:, all_features]
    
    trainX_f = dataX_features[0:len(trainX)]
    valX_f   = dataX_features[len(trainX)-len(valX):len(trainX)]
    testX_f  = dataX_features[-len(testX):]
    
    trainX_f = np.reshape(trainX_f, (trainX_f.shape[0],1, trainX_f.shape[1]))
    valX_f= np.reshape(valX_f, (valX_f.shape[0],1, valX_f.shape[1]))
    testX_f = np.reshape(testX_f, (testX_f.shape[0],1, testX_f.shape[1]))

    return trainX,valX,testX,trainY,valY,testY,trainX_f,valX_f,testX_f

In [ ]:
def HyperParameter_mapping(hyper_parametrs):
    
    learning_rate = hyper_parametrs[0]
    
    
    if hyper_parametrs[1]==0:
        denselayer_activation=None
    elif hyper_parametrs[1]==1:
        denselayer_activation="relu"
    elif hyper_parametrs[1]==2:
        denselayer_activation="linear"
    
    
    
    if hyper_parametrs[2]==0:
        output_activation=None
    elif hyper_parametrs[2]==1:
        output_activation="relu"
    elif hyper_parametrs[2]==2:
        output_activation="linear"
    
    return learning_rate,denselayer_activation,output_activation

In [ ]:
# defining hyper parameters and available range to pass it to the bayesian optimization algorithm
bounds = [{'name': 'Lag',                                     'type': 'discrete',      'domain': (5,6,7,8,9,10,11,12,13,14,15)},
          {'name': 'Learning_rate',                           'type': 'discrete',    'domain': (0.0001,0.0005,0.001,0.005,0.01,0.05)},
          {'name': 'dense_activation',                        'type': 'discrete',      'domain': (0,1,2)},
          {'name': 'output_activation',                       'type': 'discrete',      'domain': (0,1,2)},
         ]

In [ ]:
def write_to_file_ATT(file_name,lag,learning_rate,denselayer_activation,output_activation,final_smape,final_rmse,final_val_RMSE,epoch):
    fname=""+file_name +".txt"
    f1=open(fname,"a+")
    f1.write("lag=")
    f1.write(str(lag))
    f1.write(",learning_rate=")
    f1.write(str(learning_rate))
    f1.write(",denselayer_activation=")
    f1.write(str(denselayer_activation))
    f1.write(",output_activation=")
    f1.write(str(output_activation))
    f1.write(",final_smape=")
    f1.write(str(final_smape))
    f1.write(",final_rmse=")
    f1.write(str(final_rmse))
    f1.write(",final_val_RMSE=")
    f1.write(str(final_val_RMSE))
    f1.write(",epoch=")
    f1.write(str(epoch))
    f1.write("\n")
    f1.close()

In [ ]:
global_min_val_RMSE = math.inf
def Main(x):
    
    lag=int(x[:,0])
    look_forward=7
    all_features = [[4],
                    [1, 3, 6],
                    [3],
                    [0, 1, 8, 9, 10],
                    [6, 7],
                    [6],
                    [0, 3, 5, 10],
                    [0, 1, 3, 10],
                    [1],
                    [1, 5],
                    [1]
                   ]

    hyper_parametrs=x[:,1:][0]
    all_features = all_features [lag-5]
    
    
    learning_rate,denselayer_activation,output_activation=HyperParameter_mapping(hyper_parametrs)
    trainX,valX,testX,trainY,valY,testY,trainX_f,valX_f,testX_f=final_sample_creation(one_sample,features,lag,look_forward,all_features)
    
        
    test_RMSE=0
    test_SMAPE=0
    RMSE_SUM = 0
    SMAPE_SUM = 0
    val_RMSE=0
    val_RMSE_sum=0
    global global_min_val_RMSE
    input2_len=int(len(trainX_f[0][0]))
    
    for j in range(10):
        input1 = layers.Input(shape=(1,lag,))
        input2 = layers.Input(shape=(1,input2_len,))
        
        multi_head1 = MultiHeadAttention(key_dim=1,num_heads=lag,name='Multi-Head1')(input1,input1,input1)
        
        multi_head2 = MultiHeadAttention(key_dim=1,num_heads=input2_len,name='Multi-Head2')(input2,input2,input2)                    
    
        flatt1 = keras.layers.Flatten()(multi_head1)
        flatt2 = keras.layers.Flatten()(multi_head2) 
        concat=concatenate([flatt1,flatt2])
                          
        fully_connected = Dense(100,activation=denselayer_activation)(concat)

        hidden3 = Dense(look_forward,activation=output_activation)(fully_connected)

        model = Model(inputs=[input1,input2],outputs=hidden3)
        # model.summary()
        opt=tf.keras.optimizers.Adam(lr=learning_rate)
        model.compile(loss="mse",optimizer=opt,metrics=["mse"])
        count=0
        min_RMSE=0
        for k in range(500):
            history=model.fit([trainX,trainX_f],trainY,
                verbose=0,
                batch_size=1,
                )
            count+=1
           
            results = model.predict([valX,valX_f],batch_size=16, verbose=0)
           
            results = results.astype('int64')
            results = results.round()
            val_RMSE = math.sqrt(mean_squared_error(valY, results[:]))
            
            if k==0:
                min_RMSE=val_RMSE
                #print(min_RMSE)
                results = model.predict([testX,testX_f],batch_size=16, verbose=0)
                results = results.astype('int64')
                results = results.round()
                r_y=results[0: : 7]
                t_y=testY[0: : 7]
                test_RMSE = math.sqrt(mean_squared_error(t_y, r_y[:]))
                test_SMAPE = smape(t_y, r_y[:])
            if k>0 and val_RMSE<=min_RMSE:
                count=0
                min_RMSE=val_RMSE
                results = model.predict([testX,testX_f],batch_size=16, verbose=0)
                results = results.astype('int64')
                results = results.round()
                r_y=results[0: : 7]
                t_y=testY[0: : 7]
                test_RMSE = math.sqrt(mean_squared_error(t_y, r_y[:]))
                test_SMAPE = smape(t_y, r_y[:])
                
            if count>50:
                break
            
        RMSE_SUM += test_RMSE
        SMAPE_SUM += test_SMAPE
        val_RMSE_sum +=min_RMSE
        K.clear_session()
        gc.collect()
        model=None
        del history
    if math.isnan(min_RMSE) or SMAPE_SUM==0:
        final_smape=math.inf
        final_rmse=math.inf
        final_val_RMSE=math.inf
    else:
        final_smape = SMAPE_SUM/10
        final_rmse = RMSE_SUM/10
        final_val_RMSE=val_RMSE_sum/10
    print("\n\n\n Finished \n\n\n")
    print("SMAPE:\t{0} \t RMSE:\t{1}".format(final_smape, final_rmse))
    print("val_SMAPE:\t{0}".format(final_val_RMSE))
    file_name="US_att_feature_params"
    write_to_file_ATT(file_name,lag,learning_rate,denselayer_activation,output_activation,final_smape,final_rmse,final_val_RMSE,k)
    if final_val_RMSE<=global_min_val_RMSE:
        global_min_val_RMSE = final_val_RMSE
        f1=open("outputs_att_US_feature.txt","w")
        x = r_y.ravel()
        x=x/10
        for i in x:
            f1.write(str(i))
            f1.write("\n") 
    return final_val_RMSE 

In [ ]:
opt_us = GPyOpt.methods.BayesianOptimization(f=Main, domain=bounds)

In [ ]:
opt_us.run_optimization(max_iter=50)

In [ ]:
print("optimized loss: {0}".format(opt_us.fx_opt))

In [ ]:
opt_us.x_opt

In [ ]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
""".format(bounds[0]["name"],opt_us.x_opt[0],
           bounds[1]["name"],opt_us.x_opt[1],
           bounds[2]["name"],opt_us.x_opt[2],
           bounds[3]["name"],opt_us.x_opt[3],
           ))
print("optimized loss: {0}".format(opt_us.fx_opt))

In [ ]:
bounds